In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.stats as stats
import pickle
import obspy
from scipy.signal import find_peaks
import os
import pandas as pd

from sklearn.mixture import GaussianMixture
from scipy.stats import norm
from scipy.optimize import curve_fit

import scipy.integrate


In [2]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined.csv')

In [3]:
window = 2

In [4]:
def get_stf(scardec_name, wanted_type = 'fctopt'):
    db = combined[combined['scardec_name']==scardec_name]

    time = []
    momentrate = []

    event = os.listdir(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}')
    starts = [n for n, l in enumerate(event) if l.startswith(wanted_type)]
    with open(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}/{event[starts[0]]}') as f:
        lines = f.read().splitlines()

    lines = lines[2:]
    for line in lines:
        split = line.split(' ')
        split = [s for s in split if s not in ['', ' ', '\n']]
        time.append(float(split[0]))
        momentrate.append(float(split[1]))

    momentrate = np.array(momentrate)
    return momentrate, time, db

In [5]:
# Define a sum of Gaussian functions
def multi_gaussian(x, *params):
    y = np.zeros_like(x)

    for i in range(0, len(params), 3):
        amp = params[i]
        mean = params[i+1]
        stddev = params[i+2]
        y += amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))

    return y

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth


In [ ]:
def multi_gaussian(x, *params):
    y = np.zeros_like(x)
    for i in range(0, len(params), 3):
        amp = params[i]
        mean = params[i + 1]
        stddev = params[i + 2]
        y += amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))
    return y

for scardec_name in os.listdir('/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec'): #['FCTs_20170808_131949_GANSU__CHINA']:
    with open(f"/home/earthquakes1/homes/Rebecca/phd/stf/figures/large_scale_peaks/{scardec_name}.txt", "w") as file:
        file.write(f"Processing {scardec_name}\n")
        popt_list = []
        r_squared_list = []
        db = combined[combined['scardec_name']==scardec_name]
        momentrate, time, db = get_stf(scardec_name, wanted_type = 'fctmoy')
        momentrate = momentrate/1e18
        momentrate = smooth(momentrate, 1)

        not_zero = np.where(momentrate > 0)[0]

        dx = time[1]-time[0]

        start = min(not_zero)
        end = max(not_zero)
        points_before_zero = abs(min(time)/dx)

        duration = time[end] - time[start]

        start_time = time[start]
        end_time = time[end]

        total_moment = scipy.integrate.simpson(momentrate[start:end], dx = time[1]-time[0])

        for gaussian_num in range(1, 11):
            file.write(f'Fitting {gaussian_num} Gaussians\n')
            initial_guess = []
            for i in range(gaussian_num):
                initial_guess.extend([1, (max(time)/(gaussian_num+1)) * (i+1), 1])

            bounds = ([0 if i % 3 == 0 else -np.inf for i in range(len(initial_guess))],
                      [np.inf for _ in range(len(initial_guess))])
            try:
                popt, pcov = curve_fit(multi_gaussian,
                                   time,
                                   momentrate,
                                   p0=initial_guess,
                                   bounds=bounds,
                                   maxfev=5000)
            except RuntimeError:
                r_squared_list.append(0)
                popt_list.append([0, 0, 0])
                print('Optimal parameters not found')
                continue

            num_gaussians = len(popt) // 3
            params = []
            for i in range(num_gaussians):
                amp = popt[i*3]
                mean = popt[i*3 + 1]
                stddev = popt[i*3 + 2]
                params.append((amp, mean, stddev))
                x = time
                y = amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))
                file.write(f"Gaussian {i+1} - Amplitude: {amp}, Mean: {mean}, Standard Deviation: {stddev}\n")

            y_fit = multi_gaussian(time, *popt)
            residuals = momentrate - y_fit
            ss_res = np.sum(residuals**2)
            ss_tot = np.sum((momentrate - np.mean(momentrate))**2)
            r_squared = 1 - (ss_res / ss_tot)

            file.write(f'R-squared: {r_squared}\n')

            moment_gaussian = scipy.integrate.simpson(multi_gaussian(time, *popt), dx = time[1]-time[0])
            file.write(f'Proportion moment from Gaussian fit: {moment_gaussian/total_moment}\n')
            popt_list.append(popt)
            r_squared_list.append(r_squared)
            if r_squared > 0.95:
                break
        # file.write(f'{popt_list}\n')
        if r_squared_list[-1] < 0.95:
            print(scardec_name, 'r2 < 0.95')
            file.write('!!!!!!!!!!!! R-squared < 0.95 !!!!!!!!!!!!!!!!!!!!\n')
            print(popt_list)

        if len(popt_list) == 1:
            fig, axs = plt.subplots(1,1, figsize=(10, 10), sharex=True)
            for i in range(0, len(popt_list)):
                popt = popt_list[i]
                if np.array_equal(popt, [0, 0, 0]):
                    axs[j].plot(time, momentrate*1e18,
                                label='Data',
                                linestyle='-',
                                color='black',
                                zorder = 1)
                    continue
                num_gaussians = len(popt) // 3
                params = []
                for i in range(num_gaussians):
                    if i < 10:
                        ls = '--'
                    else:
                        ls = ':'
                    amp = popt[0]
                    mean = popt[1]
                    stddev = popt[2]
                    params.append((amp, mean, stddev))
                    x = time
                    y = amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))
                    axs.plot(time, y*1e18,
                                    #label=f'Gaussian {i+1}',
                                    linestyle=ls,
                                    zorder = 10)

                axs.plot(time, momentrate*1e18,
                                    label='Data',
                                    linestyle='-',
                                    color='black')
                axs.plot(time, multi_gaussian(time, *popt)*1e18,
                                        label=r'Fitted Gaussian, $R^2$ = {r_squared_list[0]:.3f}',
                                        linestyle='-',
                                        zorder = 1,
                                        color='red')
                axs.legend()
        elif len(popt_list) < 4:
            # print('in elif')
            fig, axs = plt.subplots(len(popt_list),1, figsize=(10, 10), sharex=True)
            for j in range(0, len(popt_list)):
                popt = popt_list[j]
                if np.array_equal(popt, [0, 0, 0]):
                    axs[j].plot(time, momentrate*1e18,
                                label='Data',
                                linestyle='-',
                                color='black',
                                zorder = 1)
                    continue
                num_gaussians = len(popt) // 3
                params = []
                for i in range(num_gaussians):
                    if i < 10:
                        ls = '--'
                    else:
                        ls = ':'
                    amp = popt[i*3]
                    mean = popt[i*3 + 1]
                    stddev = popt[i*3 + 2]
                    params.append((amp, mean, stddev))
                    # file.write(f"Gaussian {i+1} - Amplitude: {amp}, Mean: {mean}, Standard Deviation: {stddev}\n")
                    x = time
                    y = amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))
                    axs[j].plot(time, y*1e18,
                                    #label=f'Gaussian {i+1}',
                                    linestyle='--',
                                    zorder = 10)

                axs[j].plot(time, momentrate*1e18,
                                    label='Data',
                                    linestyle='-',
                                    color='black',
                                    zorder = 1)
                axs[j].plot(time, multi_gaussian(time, *popt_list[j])*1e18,
                                        label=fr'Sum of {num_gaussians} Gaussians, $R^2$ = {r_squared_list[j]:.3f}',
                                        linestyle='-',
                                        zorder = 12,
                                        color='red')
                axs[j].legend()
        else:
            print(len(popt_list))
            print(scardec_name)
            if len(popt_list) % 2 == 0:
                print('in if')
                fig, axs = plt.subplots(((len(popt_list) + 1) // 2) + 1, 1, figsize=(10, 10), sharex=True)
            else:
                print('in else')
                fig, axs = plt.subplots(((len(popt_list)) // 2) + 1, 1, figsize=(10, 10), sharex=True)
            # fig, axs = plt.subplots(((len(popt_list) + 1) // 2)+1, 1, figsize=(10, 10))
            for j in range(0, len(popt_list), 2):
                popt = popt_list[j]
                if np.array_equal(popt, [0, 0, 0]):
                    axs[j].plot(time, momentrate*1e18,
                                label='Data',
                                linestyle='-',
                                color='black',
                                zorder = 1)
                    continue
                num_gaussians = len(popt) // 3
                params = []
                for i in range(num_gaussians):
                    if i < 10:
                        ls = '--'
                    else:
                        ls = ':'
                    amp = popt[i*3]
                    mean = popt[i*3 + 1]
                    stddev = popt[i*3 + 2]
                    params.append((amp, mean, stddev))
                    x = time
                    y = amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))
                    axs[j//2].plot(time, y*1e18,
                                #label=f'Gaussian {i+1}',
                                linestyle=ls,
                                zorder=10)

                axs[j//2].plot(time, momentrate*1e18,
                            label='Data',
                            linestyle='-',
                            color='black',
                            zorder=1)
                axs[j//2].plot(time, multi_gaussian(time, *popt_list[j])*1e18,
                            label=fr'Sum of {num_gaussians} Gaussians, $R^2$ = {r_squared_list[j]:.3f}',
                            linestyle='-',
                            zorder=12,
                            color='red')
                axs[j//2].legend()

            # Add a new subplot for the last element in popt_list
            #fig.add_subplot(((len(popt_list) + 1) // 2) +1, 1, len(popt_list) // 2 + 1)
            #axs = fig.axes

            # Plot the last element in popt_list
            if len(popt_list) % 2 == 0:
                popt = popt_list[-1]
                num_gaussians = len(popt) // 3
                params = []
                for i in range(num_gaussians):
                    if i < 10:
                        ls = '--'
                    else:
                        ls = ':'
                    amp = popt[i*3]
                    mean = popt[i*3 + 1]
                    stddev = popt[i*3 + 2]
                    params.append((amp, mean, stddev))
                    x = time
                    y = amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))
                    axs[-1].plot(time, y*1e18,
                                #label=f'Gaussian {i+1}',
                                linestyle=ls,
                                zorder=10)

                axs[-1].plot(time, momentrate*1e18,
                            label='Data',
                            linestyle='-',
                            color='black',
                            zorder=1)
                axs[-1].plot(time, multi_gaussian(time, *popt_list[-1])*1e18,
                            label=fr'Sum of {num_gaussians} Gaussians, $R^2$ = {r_squared_list[-1]:.3f}',
                            linestyle='-',
                            zorder=12,
                            color='red')
                axs[-1].legend()
        if r_squared_list[-1] < 0.95:
            plt.suptitle(f'{scardec_name} - R-squared < 0.95')
        else:
            plt.suptitle(scardec_name)



        # Add a shared y-axis label by adding a subplot
        fig.add_subplot(111, frame_on=False)
        plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
        plt.xlabel("Time (s)")
        plt.ylabel("Moment Rate (x10^18 Nm/s)")
        plt.tight_layout()
        plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/large_scale_peaks/{scardec_name}.png')
        # if len(popt_list) > 4:
        #     plt.show()
        plt.close()


6
FCTs_19960327_205206_NEAR_COAST_OF_NICARAGUA
in if
10
FCTs_19960212_090807_SOUTH_OF_SUMBAWA__INDONESIA
in if
4
FCTs_20080415_225951_ANDREANOF_ISLANDS__ALEUTIAN_IS
in if
6
FCTs_20050505_191221_SOUTH_OF_PANAMA
in if
5
FCTs_20030907_131920_SOUTHEAST_OF_LOYALTY_ISLANDS
in else
4
FCTs_19970512_134526_PANAY__PHILIPPINES
in if
4
FCTs_20061226_122621_TAIWAN_REGION
in if
10
FCTs_20151104_034415_TIMOR_REGION
in if
4
FCTs_20190822_192711_FIJI_ISLANDS_REGION
in if
4
FCTs_19951001_155716_TURKEY
in if
5
FCTs_19951019_003206_RYUKYU_ISLANDS__JAPAN
in else
4
FCTs_20091001_015227_SOUTHERN_SUMATRA__INDONESIA
in if
4
FCTs_20181125_163732_IRAN-IRAQ_BORDER_REGION
in if
4
FCTs_20161015_080338_NEW_BRITAIN_REGION__P.N.G
in if
4
FCTs_20180821_213147_NEAR_COAST_OF_VENEZUELA
in if
4
FCTs_19990702_114531_VANCOUVER_ISLAND__CANADA_REGION
in if
5
FCTs_20030710_170637_SOUTHERN_IRAN
in else
4
FCTs_20100426_025952_SOUTHEAST_OF_TAIWAN
in if
4
FCTs_20151204_222500_SOUTHEAST_INDIAN_RIDGE
in if
4
FCTs_20050119_061136_OFF_

In [42]:
def multi_gaussian(x, *params):
    y = np.zeros_like(x)
    for i in range(0, len(params), 3):
        amp = params[i]
        mean = params[i + 1]
        stddev = params[i + 2]
        y += amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))
    return y

for scardec_name in os.listdir('/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec'): #['FCTs_20170808_131949_GANSU__CHINA']:
    with open(f"/home/earthquakes1/homes/Rebecca/phd/stf/figures/large_scale_peaks/smoothed/{scardec_name}_smoothed_10.txt", "w") as file:
        file.write(f"Processing {scardec_name}\n")
        popt_list = []
        r_squared_list = []
        db = combined[combined['scardec_name']==scardec_name]
        momentrate, time, db = get_stf(scardec_name, wanted_type = 'fctmoy')
        momentrate = momentrate/1e18
        momentrate = smooth(momentrate, 1)

        not_zero = np.where(momentrate > 0)[0]

        dx = time[1]-time[0]

        start = min(not_zero)
        end = max(not_zero)
        points_before_zero = abs(min(time)/dx)

        duration = time[end] - time[start]

        start_time = time[start]
        end_time = time[end]

        total_moment = scipy.integrate.simpson(momentrate[start:end], dx = time[1]-time[0])

        for gaussian_num in range(1, 11):
            file.write(f'Fitting {gaussian_num} Gaussians\n')
            initial_guess = []
            for i in range(gaussian_num):
                initial_guess.extend([1, (max(time)/(gaussian_num+1)) * (i+1), 1])

            bounds = ([0 if i % 3 == 0 else -np.inf for i in range(len(initial_guess))],
                      [np.inf for _ in range(len(initial_guess))])
            try:
                popt, pcov = curve_fit(multi_gaussian,
                                   time,
                                   momentrate,
                                   p0=initial_guess,
                                   bounds=bounds,
                                   maxfev=5000)
            except RuntimeError:
                r_squared_list.append(0)
                popt_list.append([0, 0, 0])
                print('Optimal parameters not found')
                continue

            num_gaussians = len(popt) // 3
            params = []
            for i in range(num_gaussians):
                amp = popt[i*3]
                mean = popt[i*3 + 1]
                stddev = popt[i*3 + 2]
                params.append((amp, mean, stddev))
                x = time
                y = amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))
                file.write(f"Gaussian {i+1} - Amplitude: {amp}, Mean: {mean}, Standard Deviation: {stddev}\n")

            y_fit = multi_gaussian(time, *popt)
            residuals = momentrate - y_fit
            ss_res = np.sum(residuals**2)
            ss_tot = np.sum((momentrate - np.mean(momentrate))**2)
            r_squared = 1 - (ss_res / ss_tot)

            file.write(f'R-squared: {r_squared}\n')

            moment_gaussian = scipy.integrate.simpson(multi_gaussian(time, *popt), dx = time[1]-time[0])
            file.write(f'Proportion moment from Gaussian fit: {moment_gaussian/total_moment}\n')
            popt_list.append(popt)
            r_squared_list.append(r_squared)
            if r_squared > 0.95:
                break
        # file.write(f'{popt_list}\n')
        if r_squared_list[-1] < 0.95:
            print(scardec_name, 'r2 < 0.95')
            file.write('!!!!!!!!!!!! R-squared < 0.95 !!!!!!!!!!!!!!!!!!!!\n')
            print(popt_list)

        if len(popt_list) == 1:
            fig, axs = plt.subplots(1,1, figsize=(10, 10), sharex=True)
            for i in range(0, len(popt_list)):
                popt = popt_list[i]
                if np.array_equal(popt, [0, 0, 0]):
                    axs[j].plot(time, momentrate*1e18,
                                label='Data',
                                linestyle='-',
                                color='black',
                                zorder = 1)
                    continue
                num_gaussians = len(popt) // 3
                params = []
                for i in range(num_gaussians):
                    if i < 10:
                        ls = '--'
                    else:
                        ls = ':'
                    amp = popt[0]
                    mean = popt[1]
                    stddev = popt[2]
                    params.append((amp, mean, stddev))
                    x = time
                    y = amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))
                    axs.plot(time, y*1e18,
                                    #label=f'Gaussian {i+1}',
                                    linestyle=ls,
                                    zorder = 10)

                axs.plot(time, momentrate*1e18,
                                    label='Data',
                                    linestyle='-',
                                    color='black')
                axs.plot(time, multi_gaussian(time, *popt)*1e18,
                                        label='Fitted Gaussian, $R^2$ = {r_squared_list[j]:.3f}',
                                        linestyle='-',
                                        zorder = 1,
                                        color='red')
                axs.legend()
        elif len(popt_list) < 4:
            # print('in elif')
            fig, axs = plt.subplots(len(popt_list),1, figsize=(10, 10), sharex=True)
            for j in range(0, len(popt_list)):
                popt = popt_list[j]
                if np.array_equal(popt, [0, 0, 0]):
                    axs[j].plot(time, momentrate*1e18,
                                label='Data',
                                linestyle='-',
                                color='black',
                                zorder = 1)
                    continue
                num_gaussians = len(popt) // 3
                params = []
                for i in range(num_gaussians):
                    if i < 10:
                        ls = '--'
                    else:
                        ls = ':'
                    amp = popt[i*3]
                    mean = popt[i*3 + 1]
                    stddev = popt[i*3 + 2]
                    params.append((amp, mean, stddev))
                    # file.write(f"Gaussian {i+1} - Amplitude: {amp}, Mean: {mean}, Standard Deviation: {stddev}\n")
                    x = time
                    y = amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))
                    axs[j].plot(time, y*1e18,
                                    #label=f'Gaussian {i+1}',
                                    linestyle='--',
                                    zorder = 10)

                axs[j].plot(time, momentrate*1e18,
                                    label='Data',
                                    linestyle='-',
                                    color='black',
                                    zorder = 1)
                axs[j].plot(time, multi_gaussian(time, *popt_list[j])*1e18,
                                        label=fr'Sum of {num_gaussians} Gaussians, $R^2$ = {r_squared_list[j]:.3f}',
                                        linestyle='-',
                                        zorder = 12,
                                        color='red')
                axs[j].legend()
        else:
            print(len(popt_list))
            print(scardec_name)
            if len(popt_list) % 2 == 0:
                print('in if')
                fig, axs = plt.subplots(((len(popt_list) + 1) // 2) + 1, 1, figsize=(10, 10), sharex=True)
            else:
                print('in else')
                fig, axs = plt.subplots(((len(popt_list)) // 2) + 1, 1, figsize=(10, 10), sharex=True)
            # fig, axs = plt.subplots(((len(popt_list) + 1) // 2)+1, 1, figsize=(10, 10))
            for j in range(0, len(popt_list), 2):
                popt = popt_list[j]
                if np.array_equal(popt, [0, 0, 0]):
                    axs[j].plot(time, momentrate*1e18,
                                label='Data',
                                linestyle='-',
                                color='black',
                                zorder = 1)
                    continue
                num_gaussians = len(popt) // 3
                params = []
                for i in range(num_gaussians):
                    if i < 10:
                        ls = '--'
                    else:
                        ls = ':'
                    amp = popt[i*3]
                    mean = popt[i*3 + 1]
                    stddev = popt[i*3 + 2]
                    params.append((amp, mean, stddev))
                    x = time
                    y = amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))
                    axs[j//2].plot(time, y*1e18,
                                #label=f'Gaussian {i+1}',
                                linestyle=ls,
                                zorder=10)

                axs[j//2].plot(time, momentrate*1e18,
                            label='Data',
                            linestyle='-',
                            color='black',
                            zorder=1)
                axs[j//2].plot(time, multi_gaussian(time, *popt_list[j])*1e18,
                            label=fr'Sum of {num_gaussians} Gaussians, $R^2$ = {r_squared_list[j]:.3f}',
                            linestyle='-',
                            zorder=12,
                            color='red')
                axs[j//2].legend()

            # Add a new subplot for the last element in popt_list
            #fig.add_subplot(((len(popt_list) + 1) // 2) +1, 1, len(popt_list) // 2 + 1)
            #axs = fig.axes

            # Plot the last element in popt_list
            if len(popt_list) % 2 == 0:
                popt = popt_list[-1]
                num_gaussians = len(popt) // 3
                params = []
                for i in range(num_gaussians):
                    if i < 10:
                        ls = '--'
                    else:
                        ls = ':'
                    amp = popt[i*3]
                    mean = popt[i*3 + 1]
                    stddev = popt[i*3 + 2]
                    params.append((amp, mean, stddev))
                    x = time
                    y = amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))
                    axs[-1].plot(time, y*1e18,
                                #label=f'Gaussian {i+1}',
                                linestyle=ls,
                                zorder=10)

                axs[-1].plot(time, momentrate*1e18,
                            label='Data',
                            linestyle='-',
                            color='black',
                            zorder=1)
                axs[-1].plot(time, multi_gaussian(time, *popt_list[-1])*1e18,
                            label=fr'Sum of {num_gaussians} Gaussians, $R^2$ = {r_squared_list[-1]:.3f}',
                            linestyle='-',
                            zorder=12,
                            color='red')
                axs[-1].legend()
        if r_squared_list[-1] < 0.95:
            plt.suptitle(f'{scardec_name} - R-squared < 0.95')
        else:
            plt.suptitle(scardec_name)



        # Add a shared y-axis label by adding a subplot
        fig.add_subplot(111, frame_on=False)
        plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
        plt.xlabel("Time (s)")
        plt.ylabel("Moment Rate (x10^18 Nm/s)")
        plt.tight_layout()
        plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/large_scale_peaks/smoothed/{scardec_name}_smoothed_10.png')
        # if len(popt_list) > 4:
        #     plt.show()
        plt.close()


6
FCTs_19960327_205206_NEAR_COAST_OF_NICARAGUA
in if
10
FCTs_19960212_090807_SOUTH_OF_SUMBAWA__INDONESIA
in if
4
FCTs_20080415_225951_ANDREANOF_ISLANDS__ALEUTIAN_IS
in if
6
FCTs_20050505_191221_SOUTH_OF_PANAMA
in if
5
FCTs_20030907_131920_SOUTHEAST_OF_LOYALTY_ISLANDS
in else
4
FCTs_19970512_134526_PANAY__PHILIPPINES
in if
4
FCTs_20061226_122621_TAIWAN_REGION
in if
10
FCTs_20151104_034415_TIMOR_REGION
in if
4
FCTs_20190822_192711_FIJI_ISLANDS_REGION
in if
4
FCTs_19951001_155716_TURKEY
in if
5
FCTs_19951019_003206_RYUKYU_ISLANDS__JAPAN
in else
4
FCTs_20091001_015227_SOUTHERN_SUMATRA__INDONESIA
in if
4
FCTs_20181125_163732_IRAN-IRAQ_BORDER_REGION
in if
4
FCTs_20161015_080338_NEW_BRITAIN_REGION__P.N.G
in if
4
FCTs_20180821_213147_NEAR_COAST_OF_VENEZUELA
in if
4
FCTs_19990702_114531_VANCOUVER_ISLAND__CANADA_REGION
in if
5
FCTs_20030710_170637_SOUTHERN_IRAN
in else
4
FCTs_20100426_025952_SOUTHEAST_OF_TAIWAN
in if
4
FCTs_20151204_222500_SOUTHEAST_INDIAN_RIDGE
in if
4
FCTs_20050119_061136_OFF_

: 